In [52]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'/opt/spark-2.4.5'

In [53]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from datetime import datetime
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize']= 20,20
%matplotlib inline
import numpy as np
import pandas as pd
 
 

In [54]:
spark = SparkSession.builder.master("local").appName("app name").config("spark.some.config.option").getOrCreate()

In [55]:
spark = SparkSession \
.builder \
.appName("GPSSpark") \
.getOrCreate()

In [56]:
s=spark.sparkContext

In [57]:
#points received on one day
sql_c=SQLContext(s)
df= sql_c.read.orc('/FYP/huntersfyp17/date=2019-11-19/part*')
 
   
 

In [58]:
df.show(5)

+--------------------+------+----------+-------------+-------------+-------------+
|           driver_id|status| timestamp|     latitude|    longitude|vehicle_model|
+--------------------+------+----------+-------------+-------------+-------------+
|130b0cb91a93f87ae...|     F|1574124726| 6.9912853241|79.8937606812|            4|
|4ef4f04a59429e5de...|     A|1574132688|6.82253980637|79.9949111938|            1|
|b95f45d71b76d0b8f...|     F|1574124314|6.72381830215|79.9069747925|            4|
|c2d85ce47df07c556...|     A|1574132577|6.89739847183|79.8562850952|            2|
|4f89a8be39b228aea...|     F|1574124780|7.32763814926|80.6758346558|            1|
+--------------------+------+----------+-------------+-------------+-------------+
only showing top 5 rows



In [59]:
trip= sql_c.read.orc('/FYP/huntersfyp17/dropdatesk=20191119/part*')

In [60]:
def getDriverTripDF(driverid):
    driver_td_df=trip.select(["tripid","driverid","pickuplatitude","pickuplongitude","droplatitude","droplongitude", "actualpickuptime","droptime","pickuptime"]).where(trip.driverid == driverid ).sort("tripid").collect()
    columns=["tripid","driverid","pickuplatitude","pickuplongitude","droplatitude","droplongitude", "actualpickuptime","droptime","pickuptime"]
    df_trip_driver = pd.DataFrame(driver_td_df, columns=columns)
    
    return df_trip_driver
    

In [61]:
def getDriverGPSpoints(driverid):
    driver_gps_df=df.select(["driver_id","timestamp","latitude","longitude"]).where(df.driver_id == driverid ).sort("timestamp").collect()    
    columns2=["driver_id","timestamp","latitude","longitude"]
    df_gps_driver = pd.DataFrame(driver_gps_df, columns=columns2)
    return df_gps_driver
    

In [62]:
#assigning relevant tripid to traces
def assign_trip_id(df_gps_driver,df_trip_driver):
    df_gps_driver['tripid']= ' '
    for index in df_gps_driver.index :
        for num in df_trip_driver.index:
            if df_trip_driver['actualpickuptime'][num] < df_gps_driver["time"][index] and df_gps_driver["time"][index]<df_trip_driver['droptime'][num]:
                df_gps_driver.loc[index, 'tripid'] =  df_trip_driver['tripid'][num]
    df_gps_driver.drop(df_gps_driver[df_gps_driver['tripid'] ==' '].index, inplace = True)
    return df_gps_driver

        

In [63]:
#distance calculation for a particular trip_id

#function to find the distance
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

 
def distance_per_trip(tripid,singleTripdf):
    Distance=0
    for num in range (len(singleTripdf)-1):
        Distance_interval= haversine_distance(singleTripdf.loc[num,'latitude'],singleTripdf.loc[num,'longitude'],singleTripdf.loc[num+1,'latitude'],singleTripdf .loc[num+1,'longitude'])
        Distance += Distance_interval
    return Distance 
def duration_per_trip(tripid,singleTripdf):  
    duration=(singleTripdf.loc[len(singleTripdf)-1,'time']-singleTripdf.loc[0,'time']).total_seconds()
    return  duration

In [64]:
#def function for a single driver 
def driver_singleday(driver_id):
    df_trip_driver=getDriverTripDF(driver_id)
    df_gps_driver=getDriverGPSpoints(driver_id) 
    for num in df_trip_driver.index:
        df_trip_driver.loc[num,'Actualduration'] = (df_trip_driver['droptime'][num]-df_trip_driver['actualpickuptime'][num]).total_seconds()
        df_trip_driver.loc[num,'WaitingTime'] = (df_trip_driver['actualpickuptime'][num]-df_trip_driver['pickuptime'][num]).total_seconds()
    
    for num in df_gps_driver.index:
        df_gps_driver.loc[num,'time'] = datetime.fromtimestamp(df_gps_driver.loc[num,'timestamp'])
    df_gps_driver['tripid']= ' '
    for index in df_gps_driver.index :
        for num in df_trip_driver.index:
            if df_trip_driver['actualpickuptime'][num] < df_gps_driver["time"][index] and df_gps_driver["time"][index]<df_trip_driver['droptime'][num]:
                df_gps_driver.loc[index, 'tripid'] =  df_trip_driver['tripid'][num]
    df_gps_driver.drop(df_gps_driver[df_gps_driver['tripid'] ==' '].index, inplace = True)
    driver_data =df_gps_driver.groupby('tripid').size()
    driver_data=driver_data.to_frame().reset_index()
    driver_data['Actualduration']=""
    driver_data['WaitingTime']=""
    for num in driver_data.index :
        for i in df_trip_driver.index:
            if driver_data['tripid'][num]== df_trip_driver['tripid'][i]:
                driver_data.loc[num,'Actualduration'] = df_trip_driver['Actualduration'][i]
                driver_data.loc[num,'WaitingTime'] = df_trip_driver['WaitingTime'][i]
                
                
    driver_data.columns=["tripid","points","Actualduration",'WaitingTime']
    driver_data["AverageP"]= driver_data["Actualduration"]/driver_data["points"]
    driver_data['distance']=''
    for i in range (len(driver_data)):
        single_trip=df_gps_driver.loc[df_gps_driver['tripid'] == driver_data['tripid'][i]]
        single_trip=single_trip.reset_index()
        driver_data.loc[i,'distance']= distance_per_trip(driver_data['tripid'][i],single_trip)
        driver_data.loc[i,'duration']= duration_per_trip(driver_data['tripid'][i],single_trip)
    driver_data.insert(0, "driver_id", driver_id)
    
    return  driver_data
 


In [65]:
path="driver.txt"
f = open(path, "r")
driverlist=f.readlines()

In [66]:
full = pd.DataFrame(columns=['driver_id','tripid','points','Actualduration','AverageP','distance','duration',"WaitingTime"])
 
count=0
for i in driverlist[0:500]:
    count+=1
    print(count)
    f= [full,driver_singleday(i[:-1])]     
    full=pd.concat(f) 


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [67]:
df_filtered_avg_points = full[full['AverageP'] <= 11]
df_filtered_avg_points = df_filtered_avg_points[df_filtered_avg_points["distance"] > 0.05]
df_filtered_avg_points = df_filtered_avg_points[df_filtered_avg_points["duration"] > 0]
df_filtered_avg_points = df_filtered_avg_points[df_filtered_avg_points["WaitingTime"] > 0]
df_filtered_avg_points = df_filtered_avg_points[df_filtered_avg_points["points"] > 0]

In [68]:
df_filtered_avg_points.to_csv('date=2019-11-19.csv', index=False)